In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [2]:
np.random.seed(1)

In [3]:
df = pd.read_csv('ibs.csv',sep= ',')
df.head()

,Group,BoxPlot Code,Study ID number,ESR(0-20 mm/hr),Sex,DoDx IBS,Duration(yr),Age(years),Height (cm),Height (SDS),...,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254,Unnamed: 255
0,Control,1,87,NaN,1,NaN,NaN,19.82,170.0,-0.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Control,1,52,NaN,1,NaN,NaN,11.61,151.5,0.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Control,1,61,NaN,0,NaN,NaN,9.59,145.4,1.47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Control,1,111,NaN,1,NaN,NaN,17.38,169.2,-0.89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Control,1,11,NaN,1,NaN,NaN,14.45,162.6,-0.47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Columns: 256 entries, Group to Unnamed: 255
dtypes: float64(241), int64(9), object(6)
memory usage: 342.1+ KB


## Data Cleaning

In [5]:
target = 'VitDStatus 30 ng/mL cut off'

### Replace empty or errors by np.nan

In [6]:
# replace empty/erros by np.nan
df = df.replace(r'^\s+$', np.nan, regex=True)
df = df.replace('\t','',regex=True)
df = df.replace(' ','',regex=True)
df = df.replace('\?',np.nan,regex=True)
df = df.replace('\<',np.nan,regex=True)
#df

### Remove outliers

In [7]:
# Drop the column `Code` as it is unrelated to the dependent variable
df = df.drop(['Vit-D draw date'],axis=1)
df.head()

,Group,BoxPlot Code,Study ID number,ESR(0-20 mm/hr),Sex,DoDx IBS,Duration(yr),Age(years),Height (cm),Height (SDS),...,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254,Unnamed: 255
0,Control,1,87,NaN,1,NaN,NaN,19.82,170.0,-0.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Control,1,52,NaN,1,NaN,NaN,11.61,151.5,0.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Control,1,61,NaN,0,NaN,NaN,9.59,145.4,1.47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Control,1,111,NaN,1,NaN,NaN,17.38,169.2,-0.89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Control,1,11,NaN,1,NaN,NaN,14.45,162.6,-0.47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Convert objects to numeric variables.

In [8]:
print(np.dtype(df['Group']))

df["Group"] = pd.to_numeric(df.Group, errors='coerce')

object


In [9]:
print(np.dtype(df['Season']))

df["Season"] = pd.to_numeric(df.Season, errors='coerce')

object


In [10]:
print(np.dtype(df['Race']))

df["Race"] = pd.to_numeric(df.Race, errors='coerce')

object


### Remove bad columns

In [11]:
def remove_bad_columns(df,bad_column_threshold):
    # find bad columns having too many missing values
    n_null = np.array(df.isnull().sum(axis=0))
    bad_col = np.array([]).astype(int)
    for i in range(len(n_null)):
        if n_null[i] >= bad_column_threshold:
            bad_col = np.append(bad_col,i)

    #print(bad_col)
    print('number of bad columns:',len(bad_col))

    # delete bad columns
    df = df.drop(df.columns[bad_col],axis=1)
    #df.info()
    return df  

In [12]:
df = remove_bad_columns(df,25)
df.info()

number of bad columns: 229
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 26 columns):
BoxPlot Code                   171 non-null int64
Study ID number                171 non-null int64
Sex                            171 non-null int64
Age(years)                     171 non-null float64
Height (cm)                    171 non-null float64
Height (SDS)                   171 non-null float64
Weight (kg)                    171 non-null float64
Weight (SDS)                   171 non-null float64
BMI                            171 non-null float64
BMI (SDS)                      171 non-null float64
BMI %ile                       167 non-null object
BMI2                           171 non-null int64
ALB                            160 non-null float64
lnALB                          160 non-null float64
ALBScore                       160 non-null float64
ALT                            163 non-null float64
lnALT                          163 non-null f

### Remove bad rows

In [13]:
 # Find rows where target is missing
def find_missing_target_rows(df,target):
    # find rows where target is missing
    missing_row = df[target].isnull()
    print('Number of rows where target are missing:')
    print(sum(missing_row))

    #df = df[~missing_row]
    missing_row_indices = np.array([t for t in range(df.shape[0]) if missing_row[t]])
    
    return missing_row_indices

In [14]:
missing_target_rows = find_missing_target_rows(df,target)

Number of rows where target are missing:
0


We find bad rows which contain too many missing values, then remove them.

In [15]:
def find_bad_rows(df,bad_row_threshold):   
    # find bad rows having too many missing values
    n_null = np.array(df.isnull().sum(axis=1))
    bad_row = np.array([]).astype(int)
    for t in range(len(n_null)):
        if n_null[t] >= bad_row_threshold:
            bad_row = np.append(bad_row,t)

    #print(bad_row)
    print('number of bad rows:',len(bad_row))

    # delete bad rows
    #df = df.drop(bad_row)
    #df.info()
    return bad_row

In [16]:
bad_rows = find_bad_rows(df,10)
df.info()

number of bad rows: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 26 columns):
BoxPlot Code                   171 non-null int64
Study ID number                171 non-null int64
Sex                            171 non-null int64
Age(years)                     171 non-null float64
Height (cm)                    171 non-null float64
Height (SDS)                   171 non-null float64
Weight (kg)                    171 non-null float64
Weight (SDS)                   171 non-null float64
BMI                            171 non-null float64
BMI (SDS)                      171 non-null float64
BMI %ile                       167 non-null object
BMI2                           171 non-null int64
ALB                            160 non-null float64
lnALB                          160 non-null float64
ALBScore                       160 non-null float64
ALT                            163 non-null float64
lnALT                          163 non-null float6

In [17]:
del_rows = np.union1d(missing_target_rows,bad_rows)
print('number of rows need to delete:',len(del_rows))

df = df.drop(del_rows)

number of rows need to delete: 0


### Separate target and attributes

In [18]:
dfx = df.drop(target,axis=1)
dfy = df[target]

### Find variable type

In [19]:
# number of uniques of each column (excluding NaN)
nu = np.array([len(pd.unique(dfx[col].dropna())) for col in dfx.columns])
print('number of uniques of each variable:')
print(nu)

number of uniques of each variable:
[  4 116   2 154 127 142 156 144 164 139  71   2  20  20   1  32  32   2
  41  41  41   2   2   2   2]


In [20]:
def define_variable_type(df,nu):
    i_binary = [] ; i_category = [] ; i_continuous = []
    for i in range(len(nu)):
        if nu[i] == 2: # binary 
            i_binary.append(i)
        elif nu[i] < 5: # !!!! NOTE: this is not always correct, depending on data
            i_category.append(i)
        else:
            i_continuous.append(i)

    print('i_binary:',i_binary)
    print('i_category:',i_category)   
    #i_binary, i_category, i_continuous
    
    variable_type  = np.ones(len(nu))  # binary
    variable_type[i_category] = 2   # categorical
    variable_type[i_continuous] = 3 # continuous

    return variable_type #,i_binary,i_category,i_continuous

In [21]:
variable_type = define_variable_type(dfx,nu)
print('variable type:',variable_type)

i_binary: [2, 11, 17, 21, 22, 23, 24]
i_category: [0, 14]
variable type: [2. 3. 1. 3. 3. 3. 3. 3. 3. 3. 3. 1. 3. 3. 2. 3. 3. 1. 3. 3. 3. 1. 1. 1.
 1.]


### Impute missing values of attributes

In [22]:
def impute_missing(df,variable_type):
    # impute binary and categorical variables by the most frequency (in each column)
    # continuous variable by median
    # input: df: pandas data frame, variable_type: list
    # output: df2: pandas data frame
    df2 = df.copy()
    for i,col in enumerate(df.columns):
        if variable_type[i] < 3: # binary or caterogy
            df2[col] = df[col].fillna(df[col].mode().iloc[0])
        else: # continuous
            df2[col] = df[col].fillna(df[col].median())    
    return df2       

In [23]:
dfx_imputed = impute_missing(dfx,variable_type)
dfx_imputed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171 entries, 0 to 170
Data columns (total 25 columns):
BoxPlot Code                 171 non-null int64
Study ID number              171 non-null int64
Sex                          171 non-null int64
Age(years)                   171 non-null float64
Height (cm)                  171 non-null float64
Height (SDS)                 171 non-null float64
Weight (kg)                  171 non-null float64
Weight (SDS)                 171 non-null float64
BMI                          171 non-null float64
BMI (SDS)                    171 non-null float64
BMI %ile                     171 non-null object
BMI2                         171 non-null int64
ALB                          171 non-null float64
lnALB                        171 non-null float64
ALBScore                     171 non-null float64
ALT                          171 non-null float64
lnALT                        171 non-null float64
ALTScore                     171 non-null float64
Vita

## Data Processing

### Attributes

In [24]:
def convert_binary_and_category(x,variable_type):
    """
    convert binary to +-1, category to one hot; remain continuous.
    """    
    onehot_encoder = OneHotEncoder(sparse=False,categories='auto')

    # create 2 initial columns
    x_new = np.zeros((x.shape[0],2))

    for i,i_type in enumerate(variable_type):
        if i_type == 1: # binary
            unique_value = np.unique(x[:,i])
            x1 = np.array([-1. if value == unique_value[0] else 1. for value in x[:,i]])        
            x_new = np.hstack((x_new,x1[:,np.newaxis]))

        elif i_type == 2: # category
            x1 = onehot_encoder.fit_transform(x[:,i].reshape(-1,1))
            x_new = np.hstack((x_new,x1))
            
        else: # continuous      
            x_new = np.hstack((x_new,x[:,i][:,np.newaxis]))      

    # drop the 2 initial column
    x_new = x_new[:,2:]
    
    return x_new.astype(float)

In [25]:
# convert x
x = np.array(dfx_imputed)
x_new = convert_binary_and_category(x,variable_type)

print(x_new.shape)
print(x_new)

(171, 28)
[[ 1.  0.  0. ...  1.  1. -1.]
 [ 1.  0.  0. ... -1.  1.  1.]
 [ 1.  0.  0. ... -1.  1.  1.]
 ...
 [ 0.  0.  0. ... -1.  1.  1.]
 [ 0.  0.  1. ...  1. -1.  1.]
 [ 0.  0.  0. ...  1. -1.  1.]]


### Target

In [26]:
y = np.array(dfy)
#print(np.unique(y,return_counts=True))

# convert taget to 0 and 1
y_new = y
#y_new = np.ones(y.shape[0])
#y_new[y =='No'] = 0

print(np.unique(y_new,return_counts=True))

(array([0, 1]), array([ 33, 138]))


In [27]:
# combine X and y and save to a file
xy_new = np.hstack((x_new,y_new[:,np.newaxis]))
np.savetxt('ibs_processed.dat',xy_new,fmt='%f')